In [4]:
import os
import sys
import pandas as pd
import dtale

import re
from bs4 import BeautifulSoup

In [5]:
file_path = r'data\email_6thJune.parquet'
df = pd.read_parquet(file_path)
print(df.shape)

(10295, 9)


In [6]:
# d = dtale.show(df)
# d.open_browser()

In [7]:
df.head(2)

,id,from_email,to_email,cc_email,subject,body,date_ist,time_ist,projectid
0,6d30273a-6d16-4c9a-b207-31261df388ba,a.cao@forell.com,samiranm@pinnacleinfotech.com,"csutradhar@pinnaclecad.com, a.lin@forell.com",RE: 23-044 Drafting,"Hi Sam,\n\nI've uploaded more markups here:\nh...",2024-05-16,05:13:05,52172
1,6b7a51f9-201f-43fa-adc2-f23203ee00d5,a.cao@forell.com,samiranm@pinnacleinfotech.com,"csutradhar@pinnaclecad.com, a.lin@forell.com",RE: 23-043 Drafting,"Sam,\n\nI've uploaded more markups here:\nhttp...",2024-05-18,03:34:35,52172


In [8]:
def remove_disclaimer_text(body):
    """
    Remove disclaimer and classification notices from the email body.

    Args:
        body (str): The body of the email.
        
    Returns:
        str: The email body without disclaimer and classification notices.
    """
    disclaimer_pattern = [
        r"\*This e-mail has been classified as.*",
        r".*classification notice.*",
        r"From:\s.*Sent:\s.*To:\s.*"
    ]
    
    for pattern in disclaimer_pattern:
        body = re.sub(pattern, '', body, flags=re.DOTALL)
    
    return body

def extract_recent_email(email_body):
    """
    Extract the most recent part of an email thread above the signature or prior email content.

    Args:
        email_body (str): The body of the email.
        
    Returns:
        str: The most recent part text.
    """
    # Use BeautifulSoup to handle HTML email bodies.
    soup = BeautifulSoup(email_body, "html.parser")
    text = soup.get_text()
    lines = text.splitlines()

    # Regex patterns for detecting common signature lines and previous messages
    signature_patterns = [
        re.compile(r'^(-{2,}|_{2,})'),  # Signatures, like "-----Original Message-----" or "__"
        re.compile(r'^\s*(Sent from my [\w\s]+|Sent with [\w\s]+)$', re.IGNORECASE), # Mobile signatures
        re.compile(r'^On \d{4}/[0-1]\d/[0-3]\d,.*'),  # Replies, like "On 2021/12/01, ... wrote:"
        re.compile(r'^\s*From:\s*.*', re.IGNORECASE), # From header
        re.compile(r'^\s*Sent:\s*.*', re.IGNORECASE), # Sent header
        re.compile(r'^To:\s*.*', re.IGNORECASE),      # To header
        re.compile(r'^Subject:\s*.*', re.IGNORECASE), # Subject header
    ]

    # Find the position of signature or previous message markers
    for i, line in enumerate(lines):
        for pattern in signature_patterns:
            if pattern.match(line):
                return "\n".join(lines[:i]).strip()  # Return everything before the signature marker

    return "\n".join(lines).strip()

def extract_text_above_signature(body, from_email):
    """
    Extract text above the signature in the email body.

    Args:
        body (str): The body of the email.
        from_email (str): The sender's email address.
        
    Returns:
        str: The extracted text above signature patterns.
    """
    username = from_email.split('@')[0]  # Extract the username
    signature_patterns = [
        r"\nfrom:\s", r"\nsent:\s", r"\nto:\s", r"\ncc:\s", r"\nsubject:\s",
        r"\nregards,\s", r"\nbest,\s", r"\nthanks,\s", r"\nsincerely,\s", r"\ncheers,\s",
        re.escape(username.lower())
    ]
    
    body_lower = body.lower()
    signature_indices = []
    
    # Find all occurrences of the signature patterns
    for pattern in signature_patterns:
        matches = list(re.finditer(pattern, body_lower, re.MULTILINE))
        signature_indices.extend([match.start() for match in matches])
    
    if signature_indices:
        signature_index = min(signature_indices)
        return body[:signature_index].strip()
    return body

def clean_email_body(body):
    """
    Clean the email body by removing unwanted text based on various regex patterns.

    Args:
        body (str): The body of the email.
    
    Returns:
        str: The cleaned email body.
    """
    patterns = {
        'urls': r'https?://\S+|www\.\S+',  # URLs
        'metadata': r'(?m)^(From|Sent|To|Cc|Bcc|Subject|Date): .*$',  # Email metadata
        'greetings': r'(?i)^(Hi|Hello|Dear|Greetings|Hey)\s+[^\n]+',  # Greetings
        'signatures': r'(?i)(Best regards|Kind regards|Regards|Cheers|Thank you|Thanks|Sincerely|Yours truly|Yours sincerely|Best|Warm regards|With regards)[^\n]+',  # Signatures
        'email_headers': r'---* Forwarded message ---*|---* Original message ---*|---* Reply Above This Line ---*',  # Email forwarding/reply headers
        'email_addresses': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Email addresses
        'non_ascii': r'[^\x00-\x7F]+',  # Non-ASCII characters
        'extra_lines': r'\n{2,}',  # Excessive newlines
        'html_tags': r'<[^>]*>',  # HTML tags
        'brackets_content': r'\[.*?\]|\{.*?\}|\<.*?\>',  # Content within brackets
        'extra_whitespace': r'\s{2,}',  # Excessive whitespace
        'unsubscribe_links': r'Unsubscribe\s+:.*|Click\s+here.*unsubscribe',  # Unsubscribe links
        'reply_lines': r'On\s+.*wrote:',  # Lines indicating the start of a reply
        'quoted_text': r'(?m)^\>.*$',  # Lines starting with ">"
        'repeated_chars': r'(.)\1{2,}',  # Repeated characters
        'generic_signature_lines': r'(?i)(^Sent from my \w+)|(--\n.*)|(Confidentiality Notice.*)',  # Generic email signatures or legal disclaimers
        'timestamps': r'\d{1,2}:\d{2}\s*(AM|PM|am|pm)?',  # Time stamps
        'dates': r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',  # Dates
        'signature_blocks': r'--\s*\n[\s\S]*',  # Signature blocks that start with -- followed by any text
        'headers_and_footers': r'(?m)^\s*-\s*$\n[\s\S]*?^\s*-\s*$',  # Headers and footers denoted by lines consisting of dashes
        'mobile_numbers': r'\b(\+?(\d{1,3})?[-. ]?)?((\(\d{1,4}\))|\d{1,4})[-. ]?\d{1,4}[-. ]?\d{1,9}\b'  # Mobile numbers
    }
    
    for key, pattern in patterns.items():
        body = re.sub(pattern, ' ', body)
    
    # Remove excess whitespace
    body = re.sub(r'\s{2,}', ' ', body)
    return body.strip()


def extract_most_recent_email_part(email_body, from_email):
    """
    Combine recent email extraction and signature pattern extraction for robust extraction.

    Args:
        email_body (str): The body of the email.
        from_email (str): The sender's email address.
        
    Returns:
        str: The extracted relevant part of the email above signatures and prior threads.
    """
    # Remove disclaimer and notice sections
    cleaned_body = remove_disclaimer_text(email_body)
    
    # Extract the most recent email section ignoring previous emails
    recent_email_content = extract_recent_email(cleaned_body)
    
    # Extract text above signatures and replies
    filtered_content_above_signature = extract_text_above_signature(recent_email_content, from_email)

    # Clean the email body
    cleaned_body = clean_email_body(filtered_content_above_signature)
    
    return filtered_content_above_signature

In [9]:
# Apply the extraction function
df['clean_email'] = df.apply(lambda row: extract_most_recent_email_part(row['body'], row['from_email']), axis=1)

C:\Users\pis05408.PINNACLE\AppData\Local\Temp\ipykernel_16228\449057951.py:33: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.



In [10]:
# d = dtale.show(test)
# d.open_browser()

### Sentiment Analysis

### NLTK

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [12]:
# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pis05408.PINNACLE\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package vader_lexicon is already up-to-date!


In [13]:
# Function to predict sentiment using VADER
def predict_sentiment_nltk(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment

In [14]:
# Apply the sentiment analysis function
df['sentiment_NLTK'] = df['clean_email'].apply(predict_sentiment_nltk)

# Extract the sentiment label and score into separate columns
df['label'] = df['sentiment_NLTK'].apply(lambda x: 'POSITIVE' if x['compound'] >= 0.05 else ('NEGATIVE' if x['compound'] <= -0.05 else 'NEUTRAL'))
df['score'] = df['sentiment_NLTK'].apply(lambda x: x['compound'])

In [15]:
print(df['label'].value_counts())

label
POSITIVE    8428
NEUTRAL     1230
NEGATIVE     637
Name: count, dtype: int64


In [16]:
# d = dtale.show(df)
# d.open_browser()

### Email Summarization

In [17]:
from transformers import pipeline

In [18]:
# # Initialize the summarization pipeline with a specific model
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# summarizer = pipeline("summarization", model="t5-large")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [19]:
# Function to summarize a single email text
def summarize_email(email_text):
    try:
        summary = summarizer(email_text, max_length=60, min_length=25, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return "Summary not available"

In [20]:
# test_df = df.head(5).copy()

In [27]:
# Apply the summarization function to each email text and create a new column
df['summary'] = df['clean_email'].apply(summarize_email)

Your max_length is set to 60, but you input_length is only 26. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 60, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Token indices sequence length is longer than the specified maximum sequence length for this model (2063 > 1024). Running this sequence through the model will result in indexing errors


Error summarizing text: index out of range in self
Error summarizing text: index out of range in self


Your max_length is set to 60, but you input_length is only 54. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Your max_length is set to 60, but you input_length is only 8. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


Error summarizing text: index out of range in self
Error summarizing text: index out of range in self


Your max_length is set to 60, but you input_length is only 52. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)
Your max_length is set to 60, but you input_length is only 57. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 60, but you input_length is only 42. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)
Your max_length is set to 60, but you input_length is only 25. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 60, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 60, but you input_length is only 43. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)
Your max_length is set to 60, but you input_length is only 58. You might consider decrea

...

Your max_length is set to 60, but you input_length is only 22. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 60, but you input_length is only 20. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_length is set to 60, but you input_length is only 31. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 60, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 60, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 60, but you input_length is only 48. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
Your max_length is set to 60, but you input_length is only 46. You might consider decrea

Error summarizing text: index out of range in self


Your max_length is set to 60, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 60, but you input_length is only 13. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 60, but you input_length is only 51. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)


Error summarizing text: index out of range in self


Your max_length is set to 60, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 60, but you input_length is only 12. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 60, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 60, but you input_length is only 16. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 60, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 60, but you input_length is only 51. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Your max_length is set to 60, but you input_length is only 26. You might consider decreasing

In [29]:
df.tail()

,id,from_email,to_email,cc_email,subject,body,date_ist,time_ist,projectid,clean_email,sentiment_NLTK,label,score,summary
10290,2d944b66-ff2b-4266-8493-e46b58dbff12,yingying.nang@koyotech.com,lacharya@pinnacleinfotech.com,"nicholas.chan@koyotech.com, michael.castro@koy...",Re: To update WDN B2 Platform and under Platfo...,"Morning Lispa, may I know the status?\n\n\nOn ...",2024-05-30,09:36:14,76396,"Morning Lispa, may I know the status?\n\n\nOn ...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NEUTRAL,0.0000,"Morning Lispa, may I know the status? Please ..."
10291,9e8cd61a-3506-401a-b982-1dcb87494e3b,zach.r@sml.ca,"ddhannawat@pinnacleinfotech.com, eric@cfp2012....","dell@sml.ca, aidan@sml.ca, alex@sml.ca, travis...",Fwd: Telus Ocean trade meeting minutes,"Morning,\n\nTelus ED subtrade meeting minutes ...",2024-04-22,20:11:17,77153,"Morning,\n\nTelus ED subtrade meeting minutes ...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NEUTRAL,0.0000,Telus ED subtrade meeting minutes attached fo...
10292,0154e208-dd0c-4f1d-ab4b-d4aaa693c4e8,zach.r@sml.ca,achaudhary@pinnacleinfotech.com,"kevin.paul@jpr.pinnaclecad.com, travis@sml.ca,...",Re: Heat Plant piping and ASHP routing,"Thanks Ansul,\n\nIf you require to go over ite...",2024-04-25,01:25:24,77237,"Thanks Ansul,\n\nIf you require to go over ite...","{'neg': 0.0, 'neu': 0.743, 'pos': 0.257, 'comp...",POSITIVE,0.6369,"If you require to go over items with us, plea..."
10293,7bcedae0-2803-4c78-ae48-1195c6eddc7c,zachary.bartula@kpff.com,arnabd@pinnacleinfotech.com,"arindamg@pinnacleinfotech.com, tadhikari@pinna...","Five Below - Austintown, OH","Hi Arnab,\n\nI have uploaded work for you on A...",2024-04-18,03:50:21,76521,"Hi Arnab,\n\nI have uploaded work for you on A...","{'neg': 0.0, 'neu': 0.827, 'pos': 0.173, 'comp...",POSITIVE,0.5859,Arnab has uploaded work for you on AWS for Fi...
10294,0ed3e9aa-4e02-4648-9e2c-c37c779b54cd,zbalhorn@triumphservices.com,sjana@pinnacleinfotech.com,"aaron@sixmocompanies.com, kislam@pinnacleinfot...",RE: TLJ Markups,"Sourav,\n\nThanks - you did a great job with t...",2024-05-23,22:40:21,78468,"Sourav,\n\nThanks - you did a great job with t...","{'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'comp...",POSITIVE,0.8910,"For column and post locations, please make su..."


In [32]:
# d = dtale.show(test_df)
# d.open_browser()

2024-06-07 15:07:55,015 - INFO     - Executing shutdown due to inactivity...
2024-06-07 15:07:59,122 - INFO     - Executing shutdown...
2024-06-07 15:07:59,128 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer


In [31]:
# Writing the data 
df.to_parquet('data/sentiment_data_output_7rdjun.parquet')

In [ ]:
# Reading the data for validation
data = pd.read_parquet('data\sentiment_data_output_3rdjun.parquet')

In [ ]:
# data.head(5)

In [ ]:
d = dtale.show(data)
d.open_browser()

2024-06-03 21:27:24,175 - INFO     - Executing shutdown due to inactivity...
2024-06-03 21:27:28,275 - INFO     - Executing shutdown...
2024-06-03 21:27:28,281 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer
